# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [26]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [27]:
# TODO: Import the necessary libs
# For example: 
import os
from dotenv import load_dotenv
from typing import List, Annotated
import chromadb
from tavily import TavilyClient
import json
from pydantic import BaseModel, Field

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from lib.evaluation import EvaluationReport
from lib.parsers import PydanticOutputParser, JsonOutputParser

In [28]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [29]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

@tool
def retrieve_game(query: str):
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 
    
    Output should be a list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    print("Retrieving game from database")   
    chroma_client = chromadb.PersistentClient(path="lib/persistent_memory")
    collection = chroma_client.get_collection("udaplay")
    retrieved_docs = collection.query(
        query_texts=query,
        n_results=2,
        include=['metadatas', 'documents', 'distances']
    )   
    return retrieved_docs['documents'][0]

#### Evaluate Retrieval Tool

In [30]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

@tool
def evaluate_retrieval(question: str, retrieved_docs: List[str]):
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    usr_prompt = "You are an excellent evaluator. You recieve an output from a retrieval tool. The output is a retrieved doc. You task is to evaluate if the documents are enough to respond to the query."\
                 f"Give a detailed explanation, so it's possible to take an action to accept it or not."\
                 f"The original questions asked by the user is: {question}"\
                 f"The document retrieved is: {retrieved_docs}"
                 
                 
    print("Evaluating output")         

    llm_judge = LLM(model="gpt-4")
    judge_response = llm_judge.invoke(
            input=usr_prompt
        )

    return judge_response.content

#### Game Web Search Tool

In [31]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

@tool
def game_web_search(question: str):
    """
    Semantic search: Finds most results in the vector DB
    args:
    - question: a question about game industry. 
    """
    print("Searching web for information")           

    client = TavilyClient(api_key=TAVILY_API_KEY)
    return client.search(question) 

### Agent

In [32]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
tools=[retrieve_game, evaluate_retrieval, game_web_search]
sys_prompt="You are an agent that answers everything about games. YOu have a set of tool. You can use the ftolls available to respond to the questions from users."\
    "First, always use the internal knowledge from the database. Always evaluate the information. If this information from the database is not enough, then use web to search for an answer."\
    "Always give a citation which helped you answer the question. Give the details of the document for the citation."

agent = Agent(
    model_name="gpt-4",
    instructions=sys_prompt,
    tools=tools
)

In [33]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
result1 = agent.invoke("When Pokémon Gold and Silver was released?", "Sess_1")
messages1 = result1.get_final_state()["messages"]

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Retrieving game from database
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
Evaluating output
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [34]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")
        
print_messages(messages1)

 -> (role = system, content = You are an agent that answers everything about games. YOu have a set of tool. You can use the ftolls available to respond to the questions from users.First, always use the internal knowledge from the database. Always evaluate the information. If this information from the database is not enough, then use web to search for an answer.Always give a citation which helped you answer the question. Give the details of the document for the citation., tool_calls = None)
 -> (role = user, content = When Pokémon Gold and Silver was released?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_ASQwAij1QS7RbyG5HCfiEdoh', function=Function(arguments='{\n"query": "Pokémon Gold and Silver release date"\n}', name='retrieve_game'), type='function')])
 -> (role = tool, content = "['[Game Boy Color] Pok\u00e9mon Gold and Silver (1999) - Second-generation Pok\u00e9mon games introducing new regions, Pok\u00e9mon, and ga

In [35]:
result2 = agent.invoke("Which one was the first 3D platformer Mario game?", "Sess_1")
messages2 = result2.get_final_state()["messages"]
print_messages(messages2)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Retrieving game from database
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
Evaluating output
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are an agent that answers everything about games. YOu have a set of tool. You can use the ftolls available to respond to the questions from users.First, always use the internal knowledge from the database. Always evaluate the information. If this information from the database is not enough, then use web to search for an answer.Always give a citation which helped you answer the question. Give the details of the document for the citation., tool_calls = None)
 -> (role = user, content = When Pokémon Gold and Silver was released?, tool_calls = None)
 -> (role = assist

In [36]:
result3 = agent.invoke("Was Mortal Kombat X realeased for Playstation 5?", "Sess_1")
messages3 = result3.get_final_state()["messages"]
print_messages(messages3)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Retrieving game from database
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
Evaluating output
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
Searching web for information
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = You are an agent that answers everything about games. YOu have a set of tool. You can use the ftolls available to respond to the questions from users.First, always use the internal knowledge from the database. Always evaluate the information. If this information from the database is not enough, then use web to search for an answer.Always give a citation which helped you answer the question. Give the details of the document for the citation., tool_cal

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes